# Flyer Mapping

## Overview

Plotting potential locations with public bulletin boards.

#### Imports

In [6]:
%load_ext autoreload
%autoreload 2

In [1]:
import re
import json
import inspect
import hashlib
import functools
from time import time
from pathlib import Path
import urllib.parse as uparse
from collections import Counter
import itertools

import requests
from pprint import pprint
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

import spacy
import gensim
import gensim.downloader
import inflect # https://github.com/jazzband/inflect
from fuzzywuzzy import fuzz

import googlemaps
import geopy
import geopy.distance as geodist

import folium
import folium.plugins
from folium.plugins import MarkerCluster,FeatureGroupSubGroup

from bing_api import Bing
from here_api import Here
from _config import config

## Data

The data comes from three main sources, each having a different means of collection and processing.

* Primary data - Potential locations (PL)
* Evaluation data - Visited locations (VL)
* Supplemental data - Open positions (OP)

#### Stage 0. Speculative list compilation
At this stage, a simple list of locations that may have public bulletin boards was compiled. Initially, the a simple mention frequency was going to be used to filter out the most likely candidates, however, it quickly became clear that most sites were reposting the same ideas from a [single source](http://www.encounter.org/files/2914/3881/8878/Top_20_places_to_find_bulletin_boards_in_a_typical_town.pdf). As such, the list was filtered down to unique values and assigned a subjective interpretation of query difficultly. From there, non-rigorous experimentation was performed to further filter down the list by searching the keyword on [google maps](https://www.google.com/maps) and observing consistency and quality of results.

#### Stage 1. Initial location discovery (PL)
Here, we make our first calls to the Google Maps API. Using a fairly large search radius of 40km, we search for each of the keywords in the list we compiled in Stage 0. This will return a maximum of $20 * \text{number of keywords}$ since that is the limit on how many nearby places will be returned with a single API call. In the case of obscure keywords or too small of a search radius, fewer than 20 results may also be presented. After completing this stage, we can now move on to real world data collection.

#### Stage 2.  Real-world collection (VL)
Now, we begin testing our hypothesized locations by visiting some of the locations collected in Stage 1. Using an app call [Map Marker](https://play.google.com/store/apps/details?id=com.exlyo.mapmarker&hl=en_US) we record coordinates, location name, location category/keyword, and the binary outcome of whether or not a bulletin board was present at that location. The app then allows us to export the collected data into an easily parsable csv format. 

#### Stage 3. Branching visited locations (PL)
Due to the fact that Gmaps' results are ordered by 'prominence' by default, we can use each of the visited locations as a new starting point for further location queries. Had we chosen to search based purely on distance, branching of the nature would be almost entirely fruitless as the results would contain far too many duplicates to be useful. This stage will be repeated each time new locations are visited.

#### Stage 4. Collecting open positions (OP)
Luckily, the CCM website provides a ready-made map of open positions which, conveniently enough, allows for export to a KML file. From there, we convert to a geojson for easy conversion into a DataFrame. The dataset provides locations exclusively in address format, so geocoding is required to get the latitude/longitude pairs we need for later plotting.  

#### Stage 5. Branching open positions (PL)
In an effort to impact the highest priority areas, we can search the areas nearby locations with active job postings. For this, we use a reduced search radius to reflect a sensible travel time and reduce duplicate results given that this is the third query of its kind. Restricting the search radius, along with the shifting origin point, forces the 'prominence' value to be recalculated, hopefully providing us diversity in the search results.

#### Locations

<details>
<summary>Locations</summary>

```
0               Grocery Store
1                     Library
2                         Gym
3       Recreational Facility
4                      Church
5                  Laundromat
6                 Coffee Shop
11           Community Center
12                 Union Hall
13               Beauty Salon
14                  Bookstore
15                 Restaurant
16                        Bar
17          Convenience Store
21                Music Store
22          Apartment complex
23                   Pharmacy
24                      Qdoba
25               Panera Bread
26             Caribou Coffee
27             Barnes & Noble
28                 Whole Food
29    Pot Belly Sandwich Shop
30                   Starbuck
31               Jimmy John's
32                     Hy-Vee
33                Post Office
36                  Town Hall
37                Barber Shop
38              Beauty Parlor
39                 Nail Salon
40            Ice Cream Stand
41                Supermarket
42                    College
43                       Mall
45                     Doctor
47                    Dentist
49                Gas Station
51           Auto Repair Shop
52            Day Care Center
53        Chamber of Commerce
54                       Bank
55               Credit Union
56             Hardware Store
57             Fitness Center
```

</details>

Sources: 

http://www.encounter.org/files/2914/3881/8878/Top_20_places_to_find_bulletin_boards_in_a_typical_town.pdf

https://www.psprint.com/resources/ultimate-list-of-places-to-distribute-your-flyers/

https://mikecooney.net/22-great-places-hang-flyers/

https://grab-its.com/tips.php

#### Globals

In [3]:
gmaps = googlemaps.Client(key=config.GMAPS_KEY, timeout=10, retry_over_query_limit=False)

In [19]:
RAD=40000 # 40000m ≈ 25 miles
METERMILE = 1609.344

In [8]:
# Subjective groupings, can be improved by instead grouping on co-occurance frequency
place_groups = {
    'food':       ['grocery_or_supermarket','bakery','food','supermarket'],
    'dining_out': ['cafe','restaurant','meal_takeaway','meal_delivery'],
    'alcohol':    ['bar', 'night_club', 'liquor_store'],
    'park_camp':  ['park','campground','amusement_park'],
    'academic':   ['library','university','school', 'book_store'],
    'health':     ['gym', 'pharmacy', 'doctor', 'dentist', 'health'],
    'beauty':     ['beauty_salon','hair_care','spa'],
    'finance':    ['finance', 'bank', 'atm','accounting'],
    'automotive': ['gas_station', 'car_repair','car_wash, gas_station','convenience_store','car_dealer'],
    'government': ['local_government_office','post_office','city_hall','courthouse'],
    'services':   ['general_contractor','laundry','real_estate_agency','florist','painter'],
    'stores':     ['electronics_store','hardware_store','department_store','clothing_store','shopping_mall','store'],
    'religious':  ['church','place_of_worship'],
    'other':      ['point_of_interest','establishment','tourist_attraction']
}
pg_rev = {x:k for k,v in place_groups.items() for x in v}

In [9]:
catcolico = [
    ('food','red','shopping-basket'),
    ('dining_out','orange','cutlery'),
    ('alcohol','purple','glass'),
    ('park_camp','darkgreen','tree'),
    ('academic','blue','graduation-cap'),
    ('health','lightred','heartbeat'),
    ('beauty','lightblue','scissors'),
    ('finance','green','dollar'),
    ('automotive','pink','car'),
    ('government','beige','gavel'),
    ('services','darkpurple','cubes'),
    ('stores','darkred','shopping-cart'),
    ('religious','lightgray','cloud'),
    ('community','cadetblue','universal-access'), # special; not in place groups
    ('other','gray','question')]

color_map = {cat:col for cat,col,_ in catcolico}
faicon_map = {cat:ico for cat,_,ico in catcolico}

In [6]:
# Removed icons: [,,,,,,,'bell',,'info-circle']
# Unused colors:  ['darkblue', ,'white','lightgreen','black'] 'cadetblue'
valid_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 
                'beige', 'darkblue', 'darkgreen', 'cadetblue','darkpurple','white',
                'pink', 'lightblue', 'lightgreen','gray', 'black', 'lightgray']

In [7]:
all_types=['accounting', 'airport', 'amusement_park', 'aquarium',
       'art_gallery', 'atm', 'bakery', 'bank', 'bar', 'beauty_salon',
       'bicycle_store', 'book_store', 'bowling_alley', 'bus_station',
       'cafe', 'campground', 'car_dealer', 'car_rental', 'car_repair',
       'car_wash', 'casino', 'cemetery', 'church', 'city_hall',
       'clothing_store', 'convenience_store', 'courthouse', 'dentist',
       'department_store', 'doctor', 'drugstore', 'electrician',
       'electronics_store', 'embassy', 'fire_station', 'florist',
       'funeral_home', 'furniture_store', 'gas_station',
       'grocery_or_supermarket', 'gym', 'hair_care', 'hardware_store',
       'hindu_temple', 'home_goods_store', 'hospital', 'insurance_agency',
       'jewelry_store', 'laundry', 'lawyer', 'library',
       'light_rail_station', 'liquor_store', 'local_government_office',
       'locksmith', 'lodging', 'meal_delivery', 'meal_takeaway', 'mosque',
       'movie_rental', 'movie_theater', 'moving_company', 'museum',
       'night_club', 'painter', 'park', 'parking', 'pet_store',
       'pharmacy', 'physiotherapist', 'plumber', 'police', 'post_office',
       'primary_school', 'real_estate_agency', 'restaurant',
       'roofing_contractor', 'rv_park', 'school', 'secondary_school',
       'shoe_store', 'shopping_mall', 'spa', 'stadium', 'storage',
       'store', 'subway_station', 'supermarket', 'synagogue',
       'taxi_stand', 'tourist_attraction', 'train_station',
       'transit_station', 'travel_agency', 'university',
       'veterinary_care', 'zoo']

Available icons:

https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css

https://fontawesome.com/v4.7.0/icons/

#### IO Functions

In [7]:
def concat_jsons(dirpath, outname):
    dirpath = Path(dirpath)
    files = [*dirpath.iterdir()]
    jlist = []
    for f in files:
        with f.open('r') as fp:
            jsf = json.load(fp)
            jsf['file_stem'] = f.stem
            jlist.append(jsf)
    json.dump(jlist,(dirpath/outname).open('w'))
    return json.load((dirpath/outname).open('r'))

def try_jsave(saveobj, fpath, no_clobber=True):
    try:
        if saveobj and fpath is not None:
            p = Path(fpath)
            if no_clobber and p.exists() and p.lstat().st_size > 0:
                p = tph.parent/(tph.stem+f'_{time():0.0f}'+tph.suffix)
                print('File exists, saving to:',p)
            with open(p,'w',encoding='utf-8') as fp: 
                json.dump(saveobj, fp, ensure_ascii=False)
    except Exception as e:
        print('ERROR:', e, e.args)
    finally:
        return saveobj

#### Utility Functions

In [113]:
HASHLOG=Path(config.paths.HASHLOG)

In [114]:
def hsfn(fn,*args,**kwargs):
    """Create a hash value for any function, parameters combination"""
    hlog = HASHLOG.read_text()
    dryrun = kwargs.pop('dryrun',False)
    bound = inspect.signature(fn).bind(*args,**kwargs)
    storeargs = {'module':fn.__module__,'name':fn.__name__,**bound.arguments}
    jstr = json.dumps(storeargs,default=str)
    
    s256 = hashlib.sha256(jstr.encode())
    hsh = s256.hexdigest()+'\n'

    if hsh not in hlog:
        HASHLOG.open(mode='a').write(hsh)
        if not dryrun:
            return fn(*args, **kwargs)
    else:
        print(f'Skipping... hashed fncall found ({hsh[:7]}...)')
        return None

Several things can go wrong with the hashing function:
1. It depends on EXACT parameter matches or it will be an entirely different hash value.
    * If the 6th decimal on a longitude/latitude value is off, new hash
    * The same is true for any changes to radius, despite undoubtedly many overlapping results.
    * Inconsistent capitalization of keywords would also cause duplication.
    * The order in which keywords are passed also matters. This can be fixed with a sort, however.
    * If 98 out of 99 destination distances overlap, but 1 differs...new entry
2. There is no 'closeness' metric, a call to `gmaps.places_nearby` with a small parameter change is no more different than a call to `gmaps.distance_matrix` or even `np.random.randint`. This could be partially alleviated by different storage files for different functions.
3. It can't account for functionally equivalent statements such as `geopy.GoogleV3.reverse` and `gmaps.reverse_geocode`
4. If the log file is corrupted or lost, all calls will need to be 'dry run' again with identical params to populate it.

Problem 1 could potentially be made slightly better by levenshtein distance of param strings prior to encoding.

A far more robust, maintainable, and likely performant solution would be to simply use a MongoDB server and pymongo.

In [70]:
def hashstore(func):
    """Works for top-level functions, otherwise not useful"""
    @functools.wraps(func)
    def wrapper_hashstore(*args, **kwargs):
        hlog = HASHLOG.read_text()
        hsh = hsfn(func,*args,**kwargs)
        if hsh not in hlog:
            HASHLOG.open(mode='a').write(hsh)
            return func(*args, **kwargs)
        else:
            print(f'hashed fncall found ({hsh[:7]}...), skipping...')
    return wrapper_hashstore

In [115]:
def partition_groups(iterable, n_groups=None, max_group_size=None):
    assert (n_groups or max_group_size) is not None, 'exactly one of `n_groups` or `max_group_size` must be provided'
    n_groups = n_groups if n_groups is not None else np.ceil(len(iterable)/(max_group_size))
    return np.array_split(iterable, n_groups)

#### Maps Api Functions

Searching for a single place by using the ID allows you to choose which [fields](https://developers.google.com/places/web-service/details#fields) are returned and costs less. This uses `gmaps.place()` if searching by id, or `gmaps.find_place()` if finding by text query.

https://developers.google.com/places/web-service/search#FindPlaceRequests

https://cloud.google.com/maps-platform/pricing/sheet/

https://developers.google.com/maps/billing/gmp-billing#data-skus

https://developers.google.com/maps/documentation/distance-matrix/intro#DistanceMatrixResponses

Based on the [gmaps billing calculator](https://mapsplatformtransition.withgoogle.com/calculator), place details is the most expensive API call for basic tier responses, costing ~$17/1000 requests. It provides little benefit over a broad area search, and should be avoided when possible.

In [149]:
def ggeocode(ggeo_func, location, fpath=None, **kwargs):
    """ggeo_func : geopy.GoogleV3.(geocode,reverse)"""
    resp = hsfn(ggeo_func, location, exactly_one=True, **kwargs)

    return resp

In [117]:
def gplaceid(query, coords, fpath=None, radius=20, store_hash=False, **kwargs):
    """Find nearest place_id that matches a given query"""
    lb = 'circle:{}@{},{}'.format(radius,*coords)
    if store_hash:
        resp = hsfn(gmaps.find_place, query, input_type='textquery', location_bias=lb, language='en', **kwargs)
    else:
        resp = gmaps.find_place(query, input_type='textquery', location_bias=lb, language='en')
    return try_jsave(resp,fpath)

In [118]:
def gpdetails_df(df, query_col, coord_col, fpath=None, radius=20, keep_fields=None, id_only=True,**kwargs):
    """Returns details or place ids of the nearest place fitting given critera 
    
    Parameters
    ----------
    df : Pandas.DataFrame, 
        DataFrame containing infomation to pass to gmaps find place API
    query_col : str, 
        Column in `df` containing query strings
    coord_col : str, 
        Column in `df` containing [latitude,longitude] coordinates
    fpath : str, (default: None)
        Path to save the json API responses
    radius : int, (default: 20)
        Search radius(meters) used to find the nearest matching result
    keep_fields : list[str], (default: ['formatted_address','geometry', 'icon', 'name',
        'permanently_closed', 'place_id', 'plus_code', 'type', 'url', 'vicinity'])
        Fields to keep from the place details response
    id_only : boolean, (default: True)
        If true, only place_ids will be returned, which is a non-billed operation
        
    Returns
    -------
    List of json resposnes or place_ids
    """
    dfm = df[[query_col,coord_col]]
    places = [gplaceid(q, co, fpath=fpath, radius=radius) for q,co in dfm.values]
    if id_only:
        return places
    else: # explict else to emphasize if and only if 
        if fpath is None:
             raise IOError('Expensive query. `fpath` before proceeding')
        if keep_fields is None:
            #'adr_address','address_component','photo','utc_offset'
            keep_fields = ['formatted_address', 'geometry', 'icon', 'name', 'permanently_closed', 
                           'place_id', 'plus_code', 'type', 'url', 'vicinity']

        results = []
        for pid in places:
            first_pid = pid['candidates'][0]['place_id']
            resp = hsfn(gmaps.place, place_id=first_pid, fields=keep_fields, language='en')
            if resp is not None:
                results.append(resp)

        return try_jsave(results,fpath)

In [119]:
def gdistances(destinations, origin, fpath, max_ele=100, **kwargs):
    """Calculate distance between up to 99 destinations from a single origin point"""
    dest_parts = partition_groups(destinations, max_group_size=(max_ele-1)) # subtract 1 for Origin
    origin = tuple(origin)
    results = []
    for part in dest_parts:
        resp = hsfn(gmaps.distance_matrix, origins=origin, destinations=part, units='imperial', language='en', **kwargs)
        if resp is not None:
            results.append(resp)

    return try_jsave(results,fpath)

In [120]:
def gdistances_df(df, dests_col, origins_col, fpath, max_ele=100, **kwargs):
    ori_dests = df.groupby(df[origins_col].astype(str))[dests_col].apply(list)
    results = [gdistances(ds, destring_list(o,' '),max_ele=max_ele) for o,ds in ori_dests.items()]
    return try_jsave(results,fpath)

In [121]:
def gsearchNB(query, coords, fpath=None, radius=40000, **kwargs):
    """Search for places near a location based on a text query. 
    Finds a maximum of 20 results per query,coords pair"""
    coords = tuple(coords)
    resp = hsfn(gmaps.places_nearby, location=coords, radius=radius, keyword=query, language='en', **kwargs)
    if resp is not None:
        resp.update({'query':query,'origin':coords,'radius':radius})
    return try_jsave(resp,fpath)

In [122]:
def gsearchNB_df(df, query_col, coord_col, fpath, radius=40000, **kwargs):
    """Search for places near locations based on a text queries using values from a DataFrame
    
    Parameters
    ----------
    df : Pandas.DataFrame, 
        DataFrame containing infomation to pass to gmaps find place API
    query_col : str, 
        Column in `df` containing query strings
    coord_col : str, 
        Column in `df` containing [latitude,longitude] coordinates
    fpath : str, (default: None)
        Path to save the json API responses
    radius : int, (default: 20)
        Search radius(meters) used to find the nearest matching result
    """
    dfm = df[[query_col,coord_col]]
    results = []
    for query,coords in dfm.values:
        coords = tuple(coords) # cast as tuple in case it is np array
        resp = hsfn(gmaps.places_nearby, location=coords, radius=radius, keyword=query, language='en', **kwargs)
        if resp is not None:
            resp.update({'query':query,'origin':coords,'radius':radius})
            results.append(resp)
            
    return try_jsave(results,fpath)

In [133]:
def gpcode(coords):
    """Long runtime when API key is not provided"""
    url_path='https://plus.codes/api?'
    params = {'address':'{},{}'.format(*coords)}
    resp = requests.get(url_path, params)
    return resp.json()

## Data Preprocessing

Prepare manually collected list of potential flyer locations.

### Keyword List

**Pipeline**:
1. Read in manually collected csv of potential flyer locations into DataFrame
2. Filter any duplicate `Places`
3. `build_kg` on DataFrame to create a knowledge graph using google's API
4. `find_named_ents` on knowledge graph DataFrame to attempt to select company names from `Places` list
5. `proc_placelist` to create query friendly results 
6. Save processed list to csv file (optionally automatic in step 5.)
7. `make_kwlist` on processed csv to preform final filtering and return clean query list
8. END

**Run Conditions**: Run once initial setup. Not run again unless new list of keywords is created.

In [123]:
def gkgsearch(query,limit=10,indent=True,apikey=None):
    b_url = 'https://kgsearch.googleapis.com'
    pth = '/v1/'
    rsc = 'entities:search'
    url_path = b_url+pth+rsc
    params = {
        'query': query,
        'limit': limit,
        'indent': indent,
        'key': apikey,
    }
    resp = requests.get(url_path, params=params)
    jresp = resp.json()
    return jresp.get('itemListElement')

In [124]:
def build_kg(queries, lim=1, apikey=None):
    df_kg = pd.concat([json_normalize(gkgsearch(q, limit=lim, apikey=key),sep='_').assign(keyword=q) 
        for q in queries], sort=False).reset_index(drop=True)

    # Clean names and drop unused columns
    df_kg = (df_kg.drop(columns=[
        '@type','result_image_contentUrl','result_image_url','result_detailedDescription_license'
    ]).rename(lambda x: re.sub(r'result_?|@|etailedD|article','',x).lower(), axis=1))
    # Re-order columns
    return df_kg[['score','keyword','name','description','type','description_body','url','description_url','id']]

In [125]:
def find_named_ents(df, min_score=5000, min_kw_ratio=85, desc_kw='company', types_filter=None):
    """Apply multiple masks to attempt to isolate named entities"""
    types_filter = set(types_filter) if types_filter is not None else set(['Organization', 'Corporation'])
    mask_score = df['score'] > min_score
    mask_kw = [fuzz.UWRatio(x,y) > min_kw_ratio for x,y in df[['keyword','name']].values]
    mask_desc = df['description'].str.contains(desc_kw,case=False)
    mask_types = df['type'].apply(lambda x: types_filter.issubset(x))
    
    return df[(mask_score & mask_kw & mask_desc & mask_types)]

In [18]:
locs_df = pd.read_csv('data/csv/flyerlocs.csv').drop_duplicates('Places')
locs_df.head()

,Places,HardQuery,Comments
0,Grocery Stores,NaN,NaN
1,Libraries,NaN,NaN
2,Gyms,NaN,NaN
3,Recreational Facilities,NaN,NaN
4,Churches,NaN,NaN


In [20]:
kg_df = build_kg(locs_df.Places, apikeykey=config.GKG_KEY); kg_df.head()

,score,keyword,name,description,type,description_body,url,description_url,id
0,29.009920,Grocery Stores,Walmart,Retail company,"[Organization, Thing, Corporation]",Walmart Inc. is an American multinational reta...,http://www.walmart.com/,https://en.wikipedia.org/wiki/Walmart,kg:/m/0841v
1,39.676849,Libraries,Greenwood Publishing Group,Publishing company,"[Organization, Thing, Corporation]",ABC-CLIO/Greenwood is an educational and acade...,NaN,https://en.wikipedia.org/wiki/Greenwood_Publis...,kg:/m/03npz5m
2,1151.340088,Gyms,ClassPass,Company,"[Thing, Corporation, Organization]",ClassPass Inc. an American company which provi...,NaN,https://en.wikipedia.org/wiki/ClassPass,kg:/m/0130gnq4
3,1.534503,Recreational Facilities,Bombardier Recreational Products,Manufacturing company,"[Corporation, Thing, Organization]",BRP Inc. is a Canadian company making various ...,http://www.brp.com,https://en.wikipedia.org/wiki/Bombardier_Recre...,kg:/m/0343nb
4,6067.500000,Coffee Shops,Tully's Coffee,Retail chain,"[Corporation, Organization, Thing, Restaurant]",Tully's Coffee is an American specialty coffee...,http://tullyscoffeeshops.com/,https://en.wikipedia.org/wiki/Tully's_Coffee,kg:/m/032716


In [22]:
find_named_ents(kg_df,min_kw_ratio=85)

,score,keyword,name,description,type,description_body,url,description_url,id
18,6114.409180,Qdoba,Qdoba,Restaurant company,"[Thing, Organization, Corporation, Restaurant]",Qdoba Mexican Eats\nis a chain of fast casual ...,http://www.qdoba.com,https://en.wikipedia.org/wiki/Qdoba,kg:/m/05rqh3
19,55637.910156,Panera Bread,Panera Bread,Bakery company,"[Restaurant, Thing, Corporation, Organization]",Panera Bread Company is an American chain stor...,http://www.panerabread.com,https://en.wikipedia.org/wiki/Panera_Bread,kg:/m/03pk18
20,10366.417969,Caribou Coffee,Caribou Coffee,Company,"[Organization, Thing, Corporation, Restaurant]",Caribou Coffee Company is an American coffee c...,NaN,https://en.wikipedia.org/wiki/Caribou_Coffee,kg:/m/03p1q_2
21,94895.757812,Barnes & Noble,Barnes &amp; Noble,Retail outlet company,"[Corporation, Organization, Thing]","Barnes &amp; Noble, Inc., is an American books...",http://www.barnesandnobleinc.com/,https://en.wikipedia.org/wiki/Barnes_%26_Noble,kg:/m/01b7dt
22,28937.314453,Whole Foods,Whole Foods Market,Supermarket company,"[Organization, LocalBusiness, Corporation, Thing]",Whole Foods Market Inc. is an American multina...,NaN,https://en.wikipedia.org/wiki/Whole_Foods_Market,kg:/m/02xf2l
23,29544.181641,Potbelly Sandwich Shop,Potbelly Sandwich Shop,Restaurant company,"[Restaurant, Thing, Corporation, Organization]",Potbelly Corporation is a publicly traded rest...,http://www.potbelly.com/,https://en.wikipedia.org/wiki/Potbelly_Sandwic...,kg:/m/05g_n0
24,25809.320312,Starbucks,Starbucks,Coffee company,"[Organization, Thing, Corporation, Restaurant]",Starbucks Corporation is an American coffee co...,http://www.starbucks.com,https://en.wikipedia.org/wiki/Starbucks,kg:/m/018c_r
25,52278.152344,Jimmy John's,Jimmy John's,Fast food restaurant company,"[Thing, Organization, Restaurant, Corporation]","Jimmy John's Franchise, LLC is an American fra...",http://www.jimmyjohns.com,https://en.wikipedia.org/wiki/Jimmy_John's,kg:/m/05pqt1
26,18910.541016,Hy-Vee,Hy-Vee,Supermarket company,"[Organization, Thing, Corporation]",Hy-Vee is a chain of more than 245 supermarket...,http://www.hy-vee.com,https://en.wikipedia.org/wiki/Hy-Vee,kg:/m/02vg5b


In [126]:
def proc_placelist(df_locs, outfile=None, ents=None):
    df = df_locs.copy()
    df['hard_query'] = df['HardQuery'].notna()
    df['named_ent'] = df['Places'].isin(ents)
    # Convert to non-entities to singular form; plural form alters query results
    ie = inflect.engine()
    df['singular'] = df.apply(lambda x: ie.singular_noun(x['Places']) if not x['named_ent'] else x['Places'],axis=1) 
    df = df[['Places','singular','named_ent','hard_query']]
    df['singular'] = df['singular'].where(df['singular'] != False, df['Places'])
    df.loc[:,['Places','singular']] = df[['Places','singular']].applymap(str.strip)

    if outfile is not None:
        df.to_csv(outfile,index=False)
    return df

In [24]:
locs_df = proc_placelist(locs_df,'data/csv/location_list.csv',find_named_ents(kg_df).keyword)

In [25]:
def make_kwlist(locs_fpath):
    df = pd.read_csv(locs_fpath)
    # Omit awkwardly phrased queries
    kwlist = locs_df['singular'].where(~(locs_df['hard_query'])).dropna().drop_duplicates()
    return kwlist

In [ ]:
qlist = make_kwlist('data/csv/location_list.csv')

### Initial Query
The first call to any API. This begins gathering results for all of the items in the list of potential establishments.

**Pipeline**:
1. (&#36;) `gsearchNB` for each keyword in the initial list from a single central location.
2. Export results of `gsearchNB` calls to json file
3. Process json into standard DataFrame format using `proc_searchNB`
4. Remove duplicate items based on `place_id` using `dedupe_placeid`
5. (&#36;) `gdistances` for each `latlong` in remain unique items
6. `add_distances` to format and insert new columnar data
7. Save DataFrame to pickle object if first run through, else append to existing master DF
8. END

**Run Conditions**: Run once initial setup. Not run again unless new list of keywords is created.

In [127]:
def update_groups(df, groupmap, types_col='types',name_col='place_name'):
    first_types = df[types_col].str[0]
    uncat_types = first_types.isin(['point_of_interest','establishment'])
    # Recreation/Community Centers
    mask_recr = uncat_types & df[name_col].str.contains(r'rec|community',case=False)
    # Chambers of Commerce
    mask_cmbr = uncat_types & df[name_col].str.contains(r'chamber',case=False)
    place_groups = first_types.map(groupmap).mask(mask_recr, 'community').mask(mask_cmbr,'government')
    return place_groups

In [186]:
def destring_list(liststr,resep=r',\s*'):
    # 6 digits of precision is accurate to ~11cm, anything further is likely noise
    # https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude
    if not isinstance(liststr,str):
        return liststr
    return np.round([*map(np.float,re.split(resep,liststr.strip('[( )]')))],6)

In [134]:
def drop_group_rename(proc_df, groupmap=None):
    df = proc_df.copy()
    start_drop = ['icon','id','photos','reference','scope','compound_code','url']
    rename_map = {'query':'keyword', 'ratings_total':'n_ratings', 'name':'place_name', 'formatted_address':'dest_addr'}
    
    df.columns = df.columns.str.split('_').str[-2:].str.join('_').str.replace('location_','').str.strip()
    df = df.drop(columns=start_drop,errors='ignore').rename(columns=rename_map)
    
    if groupmap is not None:
        df['place_group'] = update_groups(df, groupmap, types_col='types', name_col='place_name')
    df['latlong'] = df.apply(lambda x: np.round([x['lat'],x['lng']],6), axis=1)
    
    end_drop = ['lat','lng',*df.filter(like='st_l')]
    df = df.drop(columns=end_drop, errors='ignore')
    
    return df

In [135]:
def dedupe_placeid(df, masterdf_path='data/pdflyers_df.pkl', verbose=False):
    """Filter duplicate place_ids to reduce API cost"""
    df_pdf = pd.read_pickle(masterdf_path)
    dfu = df[~df['place_id'].isin(df_pdf['place_id'])].drop_duplicates('place_id')
    if verbose:
        s0 = df.shape[0]
        s1 = df.drop_duplicates('place_id').shape[0]
        s2 = df[~df['place_id'].isin(df_pdf['place_id'])].shape[0]
        s3 = dfu.shape[0]
        print('Starting samples:',s0)
        print(f'Duplicates: (Internal: {s0-s1}, External: {s0-s2})')
        print('Remaining unique entries:',s3)
        print('Yield: {:0.2%}, (Drop rate: {:0.2%})'.format((s3/s0),(s0-s3)/s0))
    return dfu

In [28]:
snb_w25k = [gsearchNB(q,config.geo.CENTERW,radius=25000) for q in qlist.str.lower()]
json.dump(snb_w25k,open('data/json/searchNB_W_allKW_r25k.json','w',encoding='utf-8'))

In [136]:
def proc_searchNB(fpath):
    with open(fpath,'r',encoding='utf-8') as fp:
        snb = json.load(fp)
    
    df = pd.concat([json_normalize(res,['results'], sep='_').assign(
            **{x:str(res[x]) for x in ['query','origin','radius']}) for res in snb],sort=False)
    
    df = drop_group_rename(df,pg_rev)
    df['origin'] = df.origin.apply(destring_list)
    
    # drop 'open_now' and rearrange columns
    df = df[['place_name', 'latlong', 'vicinity', 'keyword', 'place_group','types',
       'rating', 'n_ratings', 'place_id','global_code', 'price_level', 'origin']]
    
    return df

In [189]:
snb_df = proc_searchNB('data/json/searchNB_W_allKW_r25k.json')
snb_df = dedupe_placeid(snb_df)

In [66]:
dists_w25k = gdistances(snb_df.latlong, config.geo.CENTERW, 'data/json/distances_W_allKW_r25k.json')

In [137]:
def add_distances(df_base,dists_fpath):
    """Routing distance is calculated CENTERW -> location, geodesic distance is query origin -> location"""
    df = df_base.copy()
    with open(dists_fpath,'r',encoding='utf-8') as fp:
        distslist = json.load(fp)
    
    df['geodesic_m'] = df.apply(lambda x: geodist.geodesic(x.latlong,x.origin).meters, axis=1)
    
    dists_df = pd.DataFrame([{'dest_addr':y,'travel_m':d['distance']['value'],'travel_secs':d['duration']['value']} 
             for x in distslist for y,d in zip(x['destination_addresses'],x['rows'][0]['elements'])],df.index)
    
    dists_df['dest_addr'] = dists_df['dest_addr'].str.replace(', ?USA|United States','')
    df = pd.concat([df,dists_df],sort=False,axis=1)
    return df

In [28]:
snb_df = add_distances(snb_df,'data/json/distances_W_allKW_r25k.json')

In [196]:
pd.read_pickle('data/pdflyers_df.pkl').append(snb_df, sort=False).to_pickle('data/pdflyers_df.pkl')

### Placed Flyers

Start with raw export from the Map Markers app.

**Pipeline**:
1. Read in csv file exported from Map Marker app.
2. Process csv using `proc_visited` to format as DataFrame
3. (&#36;) `gpdetails_df` on DataFrame to gather details using each items `place_name` and `latlong`
4. Save json response to disk (performed automatically during step 3.)
5. `add_details` to concatinate new columns on to DataFrame
6. Save DataFrame to pickle object if first run through, else append to existing visited DF
7. END

**Run Conditions**: Run each time new locations are visited.

In [138]:
def proc_visited(flyer_fpath, posdf_path='data/open_pos_df.pkl'):
    """geocoder: ('here','gmaps','bing', or None)"""
    df = pd.read_csv(flyer_fpath)

    # ex. Marker 1-gas station-Holiday -> keyword:gas station, place_name:Holiday
    df[['keyword','place_name']] = df.apply(lambda x: x.Title.split('-'), result_type='expand',axis=1).iloc[:,1:]

    # append quantity where missing
    df['Description'] = df['Description'].apply(lambda x: x+'-1' if x=='Yes' else (x+'-0' if x=='No' else x))
    df['n_flyers'] = df['Description'].str.split('-').str[1].astype(np.int)
    
    df['latlong'] = df.apply(lambda x: np.round([x.Latitude,x.Longitude],6),axis=1)
    
    # Correct datetime for previously placed flyers
    fill_dts = pd.Series(['10/16/2019 11:46','10/16/2019 14:23','10/19/2019 18:12'])
    df['Timestamp'] = df['Timestamp'].mask(lambda x: x.str.split().str[0] == '11/10/2019',fill_dts).pipe(pd.to_datetime)

    # reorder and select desired columns
    df = df[['Timestamp','place_name','keyword','n_flyers', 'latlong']]
    
    df['geodesic_m'] = df['latlong'].apply(lambda x: geodist.geodesic(config.geo.CENTERW,x).meters)
    
    # filter open positions down to 100k meters from shop
    df_nbopos = pd.read_pickle(posdf_path).query('geodesic_m < 100000')
    
    mindex = df['latlong'].apply(lambda x: np.argmin([geodist.geodesic(x, v).meters for v in df_nbopos['latlong']]))
    df[['nearest_position','nearest_position_mi']] = pd.DataFrame(
        [(f"{s.Location_Host}: {s.Position}",geodist.geodesic(l,s['latlong']).miles) 
         for l,(_,s) in zip(df['latlong'],df_nbopos.iloc[mindex].iterrows())])
    
    return df

In [130]:
pd.read_csv('data/csv/placed_day2.csv').head()

,Folder name,Folder color,Latitude,Longitude,Title,Description,Color,Phone number,Timestamp
0,Day_00,ff2196f3,45.130812,-93.355416,Flyer 0.0-gas station-Kwik Trip,Yes,ff2196f3,NaN,11/10/2019 12:46
1,Day_00,ff2196f3,45.317972,-93.935816,Flyer 0.1-park-Lake Maria State Park,Yes,ff2196f3,NaN,11/10/2019 12:48
2,Day_00,ff2196f3,45.136333,-93.169280,Flyer 0.2-laundry-Maytag Laundry,Yes,ff2196f3,NaN,11/10/2019 12:50
3,NaN,ff71b300,44.985114,-93.183377,Flyer 12-college-McNeal Hall,Yes-2,ff71b300,NaN,11/16/2019 16:06
4,NaN,ff71b300,45.059031,-93.198671,Marker 13-gas station-Minnoco,No,fff44336,NaN,11/16/2019 16:58


In [207]:
df_visit_day2= proc_visited('data/csv/placed_day2.csv','data/open_pos_df.pkl')

In [208]:
df_visit_day2.head()

,Timestamp,place_name,keyword,n_flyers,latlong,geodesic_m,nearest_position,nearest_position_mi
0,2019-10-16 11:46:00,Kwik Trip,gas station,1,"[45.130812, -93.355416]",29075.875029,Anoka County Crew: Andover Field Crew Member,6.673407
1,2019-10-16 14:23:00,Lake Maria State Park,park,1,"[45.317972, -93.935816]",78569.048931,Three Rivers Crews: Central Minnesota Field Cr...,18.088296
2,2019-10-19 18:12:00,Maytag Laundry,laundry,1,"[45.136333, -93.16928]",20657.421746,Anoka County Crew: Andover Field Crew Member,8.552833
3,2019-11-16 16:06:00,McNeal Hall,college,2,"[44.985114, -93.183377]",9160.545053,Youth Outdoors Crews #1-2: Youth Outdoors Crew...,1.479060
4,2019-11-16 16:58:00,Minnoco,gas station,0,"[45.059031, -93.198671]",14584.239211,Youth Outdoors Crews #1-2: Youth Outdoors Crew...,6.147447


In [205]:
details_day2= gpdetails_df(df_visit_day2,'place_name','latlong',fpath='data/json/details_visited_day2.json',id_only=False)

In [139]:
def add_details(details_fpath, df_visited):
    with open(details_fpath, 'r', encoding='utf-8') as fp:
        df = json_normalize(json.load(fp), sep='_')
    
    df = drop_group_rename(df,pg_rev)
    df['dest_addr'] = df['dest_addr'].str.replace(', ?USA|United States','')
    df = df.combine_first(df_visited)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['n_flyers'] = df['n_flyers'].astype(np.int)
    # reorder columns
    df = df[['Timestamp','place_name','keyword','place_group','types','n_flyers','dest_addr','vicinity',
             'latlong', 'global_code','geodesic_m', 'nearest_position','nearest_position_mi', 'place_id']]
    return df

In [210]:
df_visit_details = add_details('data/json/details_visited_day2.json',df_visit_day2)
df_visit_details.to_pickle('data/visited_details_all_df.pkl')
df_visit_details.head()

,Timestamp,place_name,keyword,place_group,types,n_flyers,dest_addr,vicinity,latlong,global_code,geodesic_m,nearest_position,nearest_position_mi,place_id
0,2019-10-16 11:46:00,Kwik Trip #880,gas station,automotive,"[convenience_store, atm, gas_station, finance,...",1,"5801 96th Ave N, Brooklyn Park, MN 55443","5801 96th Avenue North, Brooklyn Park","[45.130751, -93.355403]",86Q84JJV+8R,29075.875029,Anoka County Crew: Andover Field Crew Member,6.673407,ChIJS4-HKik6s1IRU3_pSNE87kY
1,2019-10-16 14:23:00,Lake Maria State Park,park,park_camp,"[campground, tourist_attraction, lodging, park...",1,"11411 Clementa Ave NW, Monticello, MN 55362","11411 Clementa Avenue Northwest, Monticello","[45.315063, -93.951519]",86Q8828X+29,78569.048931,Three Rivers Crews: Central Minnesota Field Cr...,18.088296,ChIJVZOc2gWatFIRpusOTu6RuGs
2,2019-10-19 18:12:00,Maytag Laundry,laundry,services,"[laundry, point_of_interest, establishment]",1,"9010 Griggs Ave, Circle Pines, MN 55014","9010 Griggs Avenue, Circle Pines","[45.1363, -93.169299]",86Q84RPJ+G7,20657.421746,Anoka County Crew: Andover Field Crew Member,8.552833,ChIJy_rCW1oms1IRFFbSVWpCngA
3,2019-11-16 16:06:00,McNeal Hall,college,academic,"[university, point_of_interest, establishment]",2,"1985 Buford Ave, St Paul, MN 55108","1985 Buford Avenue, Saint Paul","[44.984688, -93.183543]",86P8XRM8+VH,9160.545053,Youth Outdoors Crews #1-2: Youth Outdoors Crew...,1.479060,ChIJqQeq7IIss1IRgIzIpO0CI4o
4,2019-11-16 16:58:00,Minnoco XPRESS,gas station,automotive,"[gas_station, point_of_interest, establishment]",0,"574 Old Hwy 8 NW, New Brighton, MN 55112","574 Old Highway 8 Northwest, New Brighton","[45.058945, -93.198574]",86Q83R52+HH,14584.239211,Youth Outdoors Crews #1-2: Youth Outdoors Crew...,6.147447,ChIJq6YIFNAus1IRNafiQLBN1oY


### Nearby Visited

**Pipeline**:
1. Read pickled DataFrame from visited locations processing stage
2. (&#36;) `gsearchNB_df` on DataFrame to find locations similar locations near each visited location. 
3. Save json response to disk (performed automatically during step 2.)
4. Process json into standard DataFrame format using `proc_searchNB`
5. Remove duplicate items based on `place_id` using `dedupe_placeid`
5. (&#36;) `gdistances` for each `latlong` in remain unique items
6. `add_distances` to format and insert new columnar data
7. Append to existing master DF
8. END

**Run Conditions**: Run after every update to visited locations

In [34]:
df_visit_details = pd.read_pickle('data/visited_details_all_df.pkl')

In [255]:
nbvisited = gsearchNB_df(df_visit_details,'keyword','latlong',fpath='data/json/searchNB_visited_day2_r25k.json',radius=25000)

In [140]:
df_nbvisit = proc_searchNB('data/json/searchNB_visited_day2_r25k.json')

In [31]:
df_nbvisit.shape, df_nbvisit.drop_duplicates('place_id').shape,df_nbvisit[~df_nbvisit.place_id.isin(df_pdf.place_id)].drop_duplicates('place_id').shape

((520, 12), (133, 12), (12, 12))

Given the growing list of places and the limited set of keywords used, we are experiencing greatly diminished returns from further queries. From an initial 520 places, our non-duplicate yield is only 12, what's more, nearly 400 were duplicates within the query itself.

In [32]:
df_nbvisit = dedupe_placeid(df_nbvisit)

In [53]:
dists_day2 = gdistances(df_nbvisit.latlong, config.geo.CENTERW,'data/json/distances_W_NBday2_r25k.json')

In [56]:
df_nbvisit = add_distances(df_nbvisit,'data/json/distances_W_NBday2_r25k.json')

In [62]:
pd.read_pickle('data/pdflyers_df.pkl').append(df_nbvisit,sort=False).to_pickle('data/pdflyers_df.pkl')

### Open Positions

**Pipeline**:
1. Collect open positions KML file from CCM website
2. Convert KML to geojson file using https://mygeodata.cloud/converter/kml-to-geojson 
3. (&#36;) Geoencode addresses to obtain latitude and longitude for each position (performed during step 4.)
4. Process geojson file using `proc_openpos` to format as DataFrame
5. Save DataFrame to pickle object if first run through, else append to existing positions DF
6. END

**Run Conditions**: Run each time the list of open positions is updated on CCM website.

In [142]:
def strip_US(address):
    rgx = re.compile(r', ?(USA|United States)')
    return rgx.sub('',address)

def geocode_df(df_geo, coords_col='latlong', addr_col='dest_addr', geocoder='here', reverse=True):
    df = df_geo.copy()
    if geocoder == 'here': 
        geoc = geopy.Here(config.HERE_APPID,config.HERE_APPCODE)
        gc_func = geoc.reverse if reverse else geoc.geocode
        addr_proc_fn = lambda x: strip_US(gc_func(x).raw['Location']['Address']['Label']) # here
    elif geocoder == 'gmaps': 
        geoc = geopy.GoogleV3(config.GMAPS_KEY)
        gc_func = geoc.reverse if reverse else geoc.geocode
        addr_proc_fn = lambda x: strip_US(ggeocode(geoc,x).address) # gmaps
    elif geocoder == 'bing': 
        geoc = geopy.Bing(config.BING_KEY)
        gc_func = geoc.reverse if reverse else geoc.geocode
        addr_proc_fn = lambda x: strip_US(gc_func(x).address) # bing
    else: 
        raise NotImplementedError("Valid options are ('here','gmaps','bing')")
    
    if not reverse:
        df[coords_col] = df[addr_col].apply(lambda x: np.round(gc_func(x).point[:2],6))
        return df
    
    df[addr_col] = df[coords_col].apply(addr_proc_fn)
    return df

In [143]:
def proc_openpos(opos_fpath, geocoder='here'):
    # Get open posisions on CCM websie
    co_geo = json.load(open(opos_fpath,'r', encoding='utf-8'))
    opgeo_df = pd.DataFrame([x['properties'] for x in co_geo['features']])
    df = geocode_df(opgeo_df,addr_col='Address', geocoder=geocoder, reverse=False)
    df['geodesic_m'] = df['latlong'].apply(lambda x: geodist.geodesic(config.geo.CENTERW,x).meters)
    return df

In [85]:
oppos_df = proc_openpos('data/geospatial/Currently_Open.csv.geojson',geocoder='gmaps')
oppos_df.to_pickle('data/open_pos_df.pkl')

In [146]:
oppos_df = pd.read_pickle('data/open_pos_df.pkl')

### Nearby positions

**Pipeline**:
1. `build_queries` using pickled DataFrame from open positions processing stage
1. (&#36;) `gsearchNB` for each keyword,coordinate pair produced by `build_queries` in the initial list from a single central location.
2. Export results of `gsearchNB` calls to json file
3. Process json into standard DataFrame format using `proc_searchNB`
4. Remove duplicate items based on `place_id` using `dedupe_placeid`
5. (&#36;) `gdistances` for each `latlong` in remain unique items
6. `add_distances` to format and insert new columnar data
7. Append to existing master DF
8. END

**Run Conditions**: Run after every update to open positions

In [145]:
def build_queries(df_openpos, max_dist=100000, keep_types=None):
    if keep_types is None:
        # 'government_office' is not a real type, 'local_government_office' resulted in worse search results
        keep_types = ['book_store','city_hall','convenience_store','gas_station', 
                      'laundry', 'library','university', 'government_office']
    
    kt_queries = [x.replace('_', ' ') for x in keep_types]
    nearpos_unq = df_openpos.drop_duplicates('Address').query('geodesic_m < @max_dist & geodesic_m > 55')['latlong']
    qmap = [(pos, ktq) for pos in nearpos_unq for ktq in kt_queries]
    return qmap

In [93]:
qmap = build_queries(oppos_df)
nbopenpos = try_jsave([gsearchNB(q, co, radius=15000) for co,q in qmap],'data/json/searchNB_opos_typeKW_r15k.json')

In [133]:
df_nbo = proc_searchNB('data/json/searchNB_opos_typeKW_r15k.json')

In [134]:
df_nbo = dedupe_placeid(df_nbo, verbose=True)

Starting samples: 1775
Duplicates: (Internal: 965, External: 1354)
Remaining unique entries: 327
Yield: 18.42%, (Drop rate: 81.58%)


In [139]:
nbo_dists = gdistances(df_nbo.latlong, config.geo.CENTERW, 'data/json/distances_opos_r15k.json')

In [142]:
df_nbo = add_distances(df_nbo,'data/json/distances_opos_r15k.json')

In [143]:
pd.read_pickle('data/pdflyers_df.pkl').append(df_nbo,sort=False).to_pickle('data/pdflyers_df.pkl')

### Group Name Analysis

In [214]:
df_pdf = pd.read_pickle('data/pdflyers_df.pkl').reset_index(drop=True)

In [236]:
def types_collocation(types_series, group_size=2, exclude_types=None, include_types=None, topn=25):
    if exclude_types is None:
        exclude_types = ['establishment','point_of_interest'] # both present in over 50% of locations
    if include_types is not None:
        types_series = types_series[types_series.apply(lambda x: any(set(include_types) & set(x)))]
    
    max_gram = types_series.apply(len).max()
    cv = CountVectorizer(ngram_range=(group_size, max_gram), stop_words=exclude_types, min_df=2)
    typecv = cv.fit_transform(types_series.str.join(' '))
    ftnames = [', '.join(x.split()) for x in cv.get_feature_names()]
    
    return pd.Series(typecv.sum(axis=0).A1,ftnames).sort_values(ascending=False)[:topn]

In [241]:
types_collocation(df_pdf.types, include_types=['local_government_office'])

city_hall, local_government_office               55
finance, local_government_office                  6
accounting, finance, local_government_office      3
accounting, finance                               3
post_office, finance, local_government_office     2
post_office, finance                              2
police, local_government_office                   2
local_government_office, park                     2
courthouse, local_government_office               2
city_hall, police, local_government_office        2
city_hall, police                                 2
dtype: int64

In [222]:
types_collocation(df_pdf.types, exclude_types=['establishment','point_of_interest','food','store'])

cafe, restaurant                                    93
convenience_store, gas_station                      71
gas_station, atm                                    63
city_hall, local_government_office                  55
gym, health                                         53
convenience_store, finance                          50
atm, finance                                        48
doctor, health                                      44
bar, restaurant                                     40
electronics_store, home_goods_store                 37
convenience_store, atm                              37
car_wash, convenience_store                         36
gas_station, convenience_store                      36
gas_station, cafe                                   35
gas_station, cafe, restaurant                       35
pharmacy, health                                    35
convenience_store, gas_station, cafe, restaurant    33
convenience_store, gas_station, cafe                33
atm, car_w

## Visualization

In [20]:
def prep_places(df, keep_cols=None):
    if keep_cols is None:
        keep_cols = ['place_name', 'latlong', 'dest_addr', 'keyword', 'types', 
                     'place_group','travel_m', 'travel_secs', 'origin']
    
    folium_df = df[keep_cols].copy()
    folium_df['miles'] = (folium_df['travel_m']/METERMILE).round(2)
    folium_df['mins'] = (folium_df['travel_secs']/60).round(1)
    folium_df = folium_df.drop(columns=['travel_m','travel_secs'])
    return folium_df

def prep_visited(df, keep_cols=None):
    if keep_cols is None:
        keep_cols = ['place_name','dest_addr','n_flyers','keyword','types','latlong','Timestamp']# date
    return df[keep_cols]

def prep_positions(df, max_dist=100000):
    return df[df['geodesic_m'] < max_dist]

In [21]:
def build_template(tinsert):
    return ("<b>{}</b><br>" # Place/Job Name
            "<i>{}</i><br>" # Adress
            +tinsert+"<br>"
            "<details><summary>{}</summary>{}</details>")

def populate_template(item,template,ins_cols=['miles','mins'], share_cols=None):
    share_cols = share_cols if share_cols is not None else ['place_name','dest_addr','keyword','types'] 
    share_cols.insert(2,ins_cols[0])
    share_cols.insert(3,ins_cols[1])

    return template.format(*(item[c] for c in share_cols[:-1]), ''.join(["<li>"+x+"</li>" for x in item[share_cols[-1]]]))

def istyle(name,subele=True):
    return f'<span style="padding-left:1em">{name}</span>' if subele else f'<span style="font-size:1.1em">{name}</span>'

In [22]:
def draw_places(df_places, m, start_enabled=None):
    if start_enabled is None:
        start_enabled = ['academic', 'services', 'other', 'community','government', 'park_camp', 'automotive']
    
    mcopts = {'maxClusterRadius':80,'spiderfyOnMaxZoom':False, 'disableClusteringAtZoom':16}
    mcg = MarkerCluster(name=istyle('Prospective Locations',False),options=mcopts)

    featgroups = {pg: FeatureGroupSubGroup(mcg,istyle(pg),show=(pg in start_enabled)) 
                  for pg in np.sort(df_places['place_group'].unique())}
    m.add_child(mcg)

    for group in list(featgroups.values()):
        m.add_child(group)

    ptemplate = build_template("<br>{:0.2f} miles ({:0.1f} mins)")

    for idx, item in df_places.iterrows():
        item_group = item['place_group']
        item_tmpl = populate_template(item,ptemplate,['miles','mins'])

        item_popup = folium.Popup(item_tmpl,max_width=150)
        item_icon = folium.Icon(color=color_map[item_group], prefix='fa', icon=faicon_map[item_group])
        mark = folium.Marker(location=item['latlong'],popup=item_popup,icon=item_icon) # tooltip=item_tooltip,

        featgroups[item_group].add_child(mark)

In [23]:
def draw_visited(df_visited, m):
    fg_vist = folium.FeatureGroup(istyle('Visited Locations',False))
    fg_yes=FeatureGroupSubGroup(fg_vist,istyle('flyer_placed'))
    fg_no=FeatureGroupSubGroup(fg_vist,istyle('no_flyer_placed'))

    m.add_child(fg_vist)
    for group in [fg_yes,fg_no]:
        m.add_child(group)

    ptemplate = build_template("<br><b>Flyers: {}</b><br>Visited: {}")
    for idx, item in df_visited.iterrows():
        item_tmpl = populate_template(item,ptemplate,['n_flyers','Timestamp'])

        # Conditional styling
        color,icon,ftgroup = ('red','close',fg_no) if item['n_flyers'] == 0 else ('green','check',fg_yes)

        item_popup = folium.Popup(item_tmpl,max_width=150)
        item_icon = folium.Icon(color=color, prefix='fa', icon=icon)
        mark = folium.Marker(location=item['latlong'],popup=item_popup,icon=item_icon)

        ftgroup.add_child(mark)

In [24]:
def draw_positions(df_openpos, m):
    fg_open = folium.FeatureGroup(istyle('Open Positions',False))
    featgroups = {k:FeatureGroupSubGroup(fg_open,istyle(k.lower().replace(' ','_'))) 
                  for k in np.sort(df_openpos['Name'].unique())}

    m.add_child(fg_open)
    for group in featgroups.values():
        m.add_child(group)

    posmap={'Youth Outdoors Crew': ['blue','child'],
            'Individual Placements': ['orange','user'],
            'Field Crews': ['green','users']}

    ptemplate = build_template('')
    for idx, item in df_openpos.iterrows():
        item_name = item['Name']
        tmpl_item = ptemplate.format(item['Position'], item['Address'],
                                     item['Location_Host'], item['Program_Description'])

        item_popup = folium.Popup(tmpl_item,max_width=150)
        item_icon = folium.Icon(color=posmap[item_name][0], prefix='fa', icon=posmap[item_name][1])
        mark = folium.Marker(location=item['latlong'], popup=item_popup, icon=item_icon)

        featgroups[item_name].add_child(mark)

In [25]:
def plot_all_markers(df_places=None, df_visited=None, df_openpos=None, outfile=None, **kwargs):
    map_center = config.geo.CENTERW
    ccm_ico = folium.CustomIcon('icons/ccm_logo.png',icon_size=(32,32))
    
    m = folium.Map(tiles=None, location=map_center, min_zoom=4)
    folium.TileLayer(control=False, min_zoom=4).add_to(m) # hide default layer from control panel
    folium.Marker(location=map_center, popup="Hamm's Shop", icon=ccm_ico).add_to(m)
    
    # Plot potential flyer locations
    if df_places is not None:
        draw_places(df_places, m)
    
    # Plot visited locations
    if df_visited is not None:
        draw_visited(df_visited, m)
    
    # Plot open positions
    if df_openpos is not None:
        draw_positions(df_openpos, m)
    
    folium.LayerControl().add_to(m)
    
    if outfile is not None:
        m.save(outfile)
    
    return m

In [2]:
df_pdfly = pd.read_pickle('data/pdflyers_df.pkl')
df_oppos = pd.read_pickle('data/open_pos_df.pkl')
df_visit = pd.read_pickle('data/visited_details_all_df.pkl')

In [27]:
mapdf_pdfly = prep_places(df_pdfly.sample(500))
mapdf_visit = prep_visited(df_visit)
mapdf_oppos = prep_positions(df_oppos)

In [28]:
plot_all_markers(mapdf_pdfly, mapdf_visit, mapdf_oppos)

## Findings

To preface this findings report, the number of observed locations is substantially lower than necessary to make any definitive claims. Additionally, bias was introduced into the locations that were visited based on travel distance and hours of operation.

**High Success**
* Universities (2/2)
* Community/Recreation Centers (3/3)
* Kwik Trip gas stations (4/4)

**Mixed Success**
* Gas stations general (4/11)
* Laundromats (1/4)

**Limited/No Success**
* Speedway gas stations (0/3)* (one location not recorded)